**Problem-Solving as Search**

The 8-puzzle problem involves a 3x3 grid with 8 numbered tiles and one empty space. The goal is to rearrange the tiles to achieve a specific goal state. In this implementation, both best-first and A* search algorithms are used to find a solution path from a given initial state to the goal state.

Heuristics used:
*   Euclidean Distance: This heuristic calculates the Euclidean distance between the current position of each tile and its goal position.
*   Manhattan Distance: This heuristic calculates the Manhattan distance between the current position of each tile and its goal position.
*   Misplaced Tiles: This heuristic counts the number of tiles that are not in their goal position.





In [4]:
import random
import math

MAX_MOVE_COUNT = 3000
GOAL_STATE = [[1, 2, 3],
              [4, 5, 6],
              [7, 8, 0]]
step_count = [[0, 0, 0], [0, 0, 0]]


def find_index(item, seq):
    if item in seq:
        return seq.index(item)
    else:
        return -1


class EightPuzzle:

    def __init__(self):
        self._hval = 0
        self._depth = 0
        self._parent = None
        self.adj_matrix = []
        for i in range(3):
            self.adj_matrix.append(GOAL_STATE[i][:])

    def __eq__(self, other):
        if self.__class__ != other.__class__:
            return False
        else:
            return self.adj_matrix == other.adj_matrix

    def __str__(self):
        res = ''
        for row in range(3):
            res += ' '.join(map(str, self.adj_matrix[row]))
            res += '\r\n'
        return res

    def _clone(self):
        p = EightPuzzle()
        for i in range(3):
            p.adj_matrix[i] = self.adj_matrix[i][:]
        return p

    def _get_legal_moves(self):
        row, col = self.find(0)
        free = []

        if row > 0:
            free.append((row - 1, col))
        if col > 0:
            free.append((row, col - 1))
        if row < 2:
            free.append((row + 1, col))
        if col < 2:
            free.append((row, col + 1))

        return free

    def _generate_moves(self):
        free = self._get_legal_moves()
        zero = self.find(0)

        def swap_and_clone(a, b):
            p = self._clone()
            p.swap(a, b)
            p._depth = self._depth + 1
            p._parent = self
            return p

        return map(lambda pair: swap_and_clone(zero, pair), free)

    def _generate_solution_path(self, path):
        if self._parent is None:
            return path
        else:
            path.append(self)
            return self._parent._generate_solution_path(path)

    def solve_using_astar(self, heuristic):
        def is_solved(puzzle):
            return puzzle.adj_matrix == GOAL_STATE

        open_list = [self]
        closed_list = []
        move_count = 0
        while len(open_list) > 0:
            x = open_list.pop(0)
            move_count += 1

            if move_count > MAX_MOVE_COUNT:
                print("Maximum move count reached!!")
                return [], move_count

            if is_solved(x):
                if len(closed_list) > 0:
                    return x._generate_solution_path([]), move_count
                else:
                    return [x]

            succ = x._generate_moves()
            idx_open = idx_closed = -1
            for move in succ:
                idx_open = find_index(move, open_list)
                idx_closed = find_index(move, closed_list)
                hval = heuristic(move)
                fval = hval + move._depth

                if idx_closed == -1 and idx_open == -1:
                    move._hval = hval
                    open_list.append(move)
                elif idx_open > -1:
                    copy = open_list[idx_open]
                    if fval < copy._hval + copy._depth:
                        copy._hval = hval
                        copy._parent = move._parent
                        copy._depth = move._depth
                elif idx_closed > -1:
                    copy = closed_list[idx_closed]
                    if fval < copy._hval + copy._depth:
                        move._hval = hval
                        closed_list.remove(copy)
                        open_list.append(move)

            closed_list.append(x)
            open_list = sorted(open_list, key=lambda p: p._hval + p._depth)

        return [], move_count

    def solve_using_best_first_search(self, heuristic):
        def is_solved(puzzle):
            return puzzle.adj_matrix == GOAL_STATE

        open_list = [self]
        closed_list = []
        move_count = 0
        while len(open_list) > 0:
            x = open_list.pop(0)
            move_count += 1

            if move_count > MAX_MOVE_COUNT:
                print("Maximum move count reached!!")
                return [], move_count

            if is_solved(x):
                if len(closed_list) > 0:
                    return x._generate_solution_path([]), move_count
                else:
                    return [x]

            succ = x._generate_moves()
            idx_open = idx_closed = -1
            for move in succ:
                idx_open = find_index(move, open_list)
                idx_closed = find_index(move, closed_list)
                hval = heuristic(move)
                fval = hval

                if idx_closed == -1 and idx_open == -1:
                    move._hval = hval
                    open_list.append(move)
                elif idx_open > -1:
                    copy = open_list[idx_open]
                    if fval < copy._hval:
                        copy._hval = hval
                        copy._parent = move._parent
                elif idx_closed > -1:
                    copy = closed_list[idx_closed]
                    if fval < copy._hval:
                        move._hval = hval
                        closed_list.remove(copy)
                        open_list.append(move)

            closed_list.append(x)
            open_list = sorted(open_list, key=lambda p: p._hval)

        return [], move_count

    def initialize_matrix(self, values):
        i = 0
        for row in range(3):
            for col in range(3):
                self.adj_matrix[row][col] = int(values[i])
                i += 1

    def find(self, value):
        if value < 0 or value > 8:
            raise Exception("Value out of range")

        for row in range(3):
            for col in range(3):
                if self.adj_matrix[row][col] == value:
                    return row, col

    def get_value(self, row, col):
        return self.adj_matrix[row][col]

    def set_value(self, row, col, value):
        self.adj_matrix[row][col] = value

    def swap(self, pos_a, pos_b):
        temp = self.get_value(*pos_a)
        self.set_value(pos_a[0], pos_a[1], self.get_value(*pos_b))
        self.set_value(pos_b[0], pos_b[1], temp)


def calculate_heuristic(puzzle, item_total_calc, total_calc):
    t = 0
    for row in range(3):
        for col in range(3):
            val = puzzle.get_value(row, col) - 1
            target_col = val % 3
            target_row = val // 3

            if target_row < 0:
                target_row = 2

            t += item_total_calc(row, target_row, col, target_col)

    return total_calc(t)

def heuristic_euclidean_distance(puzzle):
    return calculate_heuristic(puzzle,
                                lambda r, tr, c, tc: math.sqrt((tr - r)**2 + (tc - c)**2),
                                lambda t: t)

def heuristic_manhattan_distance(puzzle):
    return calculate_heuristic(puzzle,
                                lambda r, tr, c, tc: abs(tr - r) + abs(tc - c),
                                lambda t: t)

def heuristic_misplaced_tiles(puzzle):
    return calculate_heuristic(puzzle,
                                lambda r, tr, c, tc: misplaced_tiles_count(puzzle),
                                lambda t: t)


def misplaced_tiles_count(puzzle):
    count = 0
    for row in range(3):
        for col in range(3):
            if puzzle.get_value(row, col) != GOAL_STATE[row][col]:
                count += 1
    return count


def main():
    input_matrix = input("\nEnter the initial matrix state(For example: 457812360): ")
    p = EightPuzzle()
    p.initialize_matrix(input_matrix)
    perform_search(p)


def perform_search(p):
    print("\n\nBreadth first Search")
    print("\n\nHeuristic 1: Euclidean distance")
    print(p)
    path, count = p.solve_using_best_first_search(heuristic_euclidean_distance)
    if path != []:
        path.reverse()
        for i in path:
            print(i.adj_matrix)
        step_count[1][1] = step_count[1][1] + len(path)
        print("\nAverage Steps: ", len(path))
    else:
        print("Search aborted after exploring", count - 1, "states")

    print("\n\nHeuristic 2: Manhattan distance ")
    print(p)
    path, count = p.solve_using_best_first_search(heuristic_manhattan_distance)
    if path != []:
        path.reverse()
        for i in path:
            print(i.adj_matrix)
        step_count[1][0] = step_count[1][0] + len(path)
        print("\nAverage Steps: ", len(path))
    else:
        print("Search aborted after exploring", count - 1, "states")

    print("\n\nHeuristic 3: Misplaced tiles")
    print(p)
    path, count = p.solve_using_best_first_search(heuristic_misplaced_tiles)
    if path != []:
        path.reverse()
        for i in path:
            print(i.adj_matrix)
        step_count[1][2] = step_count[1][2] + len(path)
        print("\nAverage Steps: ", len(path))
    else:
        print("Search aborted after exploring", count - 1, "states")
    print("\n\n A* Search ")
    print("\n\nHeuristic 1: Euclidean distance ")
    print(p)
    path, count = p.solve_using_astar(heuristic_euclidean_distance)
    if path != []:
        path.reverse()
        for i in path:
            print(i.adj_matrix)
        step_count[0][1] = step_count[0][1] + len(path)
        print("\nAverage Steps: ", len(path))
    else:
        print("\nSearch aborted after exploring", count - 1, "states")

    print("\n\nHeuristic 2: Manhattan distance ")
    print(p)
    path, count = p.solve_using_astar(heuristic_manhattan_distance)
    if path != []:
        path.reverse()
        for i in path:
            print(i.adj_matrix)
        step_count[1][0] = step_count[0][0] + len(path)
        print("\nAverage Steps: ", len(path))
    else:
        print("\nSearch aborted after exploring", count - 1, "states")

    print("\n\nHeuristic 3: Misplaced tiles ")
    print(p)
    path, count = p.solve_using_astar(heuristic_misplaced_tiles)
    if path != []:
        path.reverse()
        for i in path:
            print(i.adj_matrix)
        step_count[0][2] = step_count[0][2] + len(path)
        print("\nAverage Steps: ", len(path))
    else:
        print("\nSearch aborted after exploring", count - 1, "states")

if __name__ == "__main__":
    main()



Enter the initial matrix state(For example: 457812360): 132405687


Breadth first Search


Heuristic 1: Euclidean distance
1 3 2
4 0 5
6 8 7

[[1, 3, 2], [4, 5, 0], [6, 8, 7]]
[[1, 3, 0], [4, 5, 2], [6, 8, 7]]
[[1, 0, 3], [4, 5, 2], [6, 8, 7]]
[[1, 5, 3], [4, 0, 2], [6, 8, 7]]
[[1, 5, 3], [4, 2, 0], [6, 8, 7]]
[[1, 5, 3], [4, 2, 7], [6, 8, 0]]
[[1, 5, 3], [4, 2, 7], [6, 0, 8]]
[[1, 5, 3], [4, 2, 7], [0, 6, 8]]
[[1, 5, 3], [0, 2, 7], [4, 6, 8]]
[[1, 5, 3], [2, 0, 7], [4, 6, 8]]
[[1, 5, 3], [2, 7, 0], [4, 6, 8]]
[[1, 5, 3], [2, 7, 8], [4, 6, 0]]
[[1, 5, 3], [2, 7, 8], [4, 0, 6]]
[[1, 5, 3], [2, 0, 8], [4, 7, 6]]
[[1, 5, 3], [2, 8, 0], [4, 7, 6]]
[[1, 5, 3], [2, 8, 6], [4, 7, 0]]
[[1, 5, 3], [2, 8, 6], [4, 0, 7]]
[[1, 5, 3], [2, 0, 6], [4, 8, 7]]
[[1, 5, 3], [0, 2, 6], [4, 8, 7]]
[[1, 5, 3], [4, 2, 6], [0, 8, 7]]
[[1, 5, 3], [4, 2, 6], [8, 0, 7]]
[[1, 5, 3], [4, 2, 6], [8, 7, 0]]
[[1, 5, 3], [4, 2, 0], [8, 7, 6]]
[[1, 5, 3], [4, 0, 2], [8, 7, 6]]
[[1, 5, 3], [4, 7, 2], [8, 0, 6]]
[[1, 5,

**15-puzzle**

In [8]:
import random
import math

MAX_MOVE_COUNT = 5000
GOAL_STATE = [[1, 2, 3, 4],
              [5, 6, 7, 8],
              [9, 10, 11, 12],
              [13, 14, 15, 0]]
step_count = [[0, 0, 0], [0, 0, 0]]


class FifteenPuzzle:

    def __init__(self):
        self._hval = 0
        self._depth = 0
        self._parent = None
        self.adj_matrix = []
        for i in range(4):
            self.adj_matrix.append(GOAL_STATE[i][:])

    def __eq__(self, other):
        if self.__class__ != other.__class__:
            return False
        else:
            return self.adj_matrix == other.adj_matrix

    def __str__(self):
        res = ''
        for row in range(4):
            res += ' '.join(map(str, self.adj_matrix[row]))
            res += '\r\n'
        return res

    def _clone(self):
        p = FifteenPuzzle()
        for i in range(4):
            p.adj_matrix[i] = self.adj_matrix[i][:]
        return p

    def _get_legal_moves(self):
        row, col = self.find(0)
        free = []

        if row > 0:
            free.append((row - 1, col))
        if col > 0:
            free.append((row, col - 1))
        if row < 3:
            free.append((row + 1, col))
        if col < 3:
            free.append((row, col + 1))

        return free

    def _generate_moves(self):
        free = self._get_legal_moves()
        zero = self.find(0)

        def swap_and_clone(a, b):
            p = self._clone()
            p.swap(a, b)
            p._depth = self._depth + 1
            p._parent = self
            return p

        return map(lambda pair: swap_and_clone(zero, pair), free)

    def _generate_solution_path(self, path):
        if self._parent is None:
            return path
        else:
            path.append(self)
            return self._parent._generate_solution_path(path)

    def solve_using_astar(self, heuristic):
        def is_solved(puzzle):
            return puzzle.adj_matrix == GOAL_STATE

        open_list = [self]
        closed_list = []
        move_count = 0
        while len(open_list) > 0:
            x = open_list.pop(0)
            move_count += 1

            if move_count > MAX_MOVE_COUNT:
                print("Maximum move count reached!!")
                return [], move_count

            if is_solved(x):
                if len(closed_list) > 0:
                    return x._generate_solution_path([]), move_count
                else:
                    return [x]

            succ = x._generate_moves()
            idx_open = idx_closed = -1
            for move in succ:
                idx_open = find_index(move, open_list)
                idx_closed = find_index(move, closed_list)
                hval = heuristic(move)
                fval = hval + move._depth

                if idx_closed == -1 and idx_open == -1:
                    move._hval = hval
                    open_list.append(move)
                elif idx_open > -1:
                    copy = open_list[idx_open]
                    if fval < copy._hval + copy._depth:
                        copy._hval = hval
                        copy._parent = move._parent
                        copy._depth = move._depth
                elif idx_closed > -1:
                    copy = closed_list[idx_closed]
                    if fval < copy._hval + copy._depth:
                        move._hval = hval
                        closed_list.remove(copy)
                        open_list.append(move)

            closed_list.append(x)
            open_list = sorted(open_list, key=lambda p: p._hval + p._depth)

        return [], move_count

    def solve_using_best_first_search(self, heuristic):
        def is_solved(puzzle):
            return puzzle.adj_matrix == GOAL_STATE

        open_list = [self]
        closed_list = []
        move_count = 0
        while len(open_list) > 0:
            x = open_list.pop(0)
            move_count += 1

            if move_count > MAX_MOVE_COUNT:
                print("Maximum move count reached!!")
                return [], move_count

            if is_solved(x):
                if len(closed_list) > 0:
                    return x._generate_solution_path([]), move_count
                else:
                    return [x]

            succ = x._generate_moves()
            idx_open = idx_closed = -1
            for move in succ:
                idx_open = find_index(move, open_list)
                idx_closed = find_index(move, closed_list)
                hval = heuristic(move)
                fval = hval

                if idx_closed == -1 and idx_open == -1:
                    move._hval = hval
                    open_list.append(move)
                elif idx_open > -1:
                    copy = open_list[idx_open]
                    if fval < copy._hval:
                        copy._hval = hval
                        copy._parent = move._parent
                elif idx_closed > -1:
                    copy = closed_list[idx_closed]
                    if fval < copy._hval:
                        move._hval = hval
                        closed_list.remove(copy)
                        open_list.append(move)

            closed_list.append(x)
            open_list = sorted(open_list, key=lambda p: p._hval)

        return [], move_count

    def initialize_matrix(self, values):
        i = 0
        for row in range(4):
            for col in range(4):
                self.adj_matrix[row][col] = int(values[i])
                i += 1

    def find(self, value):
        if value < 0 or value > 15:
            raise Exception("Value out of range")

        for row in range(4):
            for col in range(4):
                if self.adj_matrix[row][col] == value:
                    return row, col

    def get_value(self, row, col):
        return self.adj_matrix[row][col]

    def set_value(self, row, col, value):
        self.adj_matrix[row][col] = value

    def swap(self, pos_a, pos_b):
        temp = self.get_value(*pos_a)
        self.set_value(pos_a[0], pos_a[1], self.get_value(*pos_b))
        self.set_value(pos_b[0], pos_b[1], temp)


def calculate_heuristic(puzzle, item_total_calc, total_calc):
    t = 0
    for row in range(4):
        for col in range(4):
            val = puzzle.get_value(row, col) - 1
            target_col = val % 4
            target_row = val // 4

            if target_row < 0:
                target_row = 3

            t += item_total_calc(row, target_row, col, target_col)

    return total_calc(t)


def heuristic_euclidean_distance(puzzle):
    return calculate_heuristic(puzzle,
                                lambda r, tr, c, tc: math.sqrt((tr - r)**2 + (tc - c)**2),
                                lambda t: t)


def heuristic_manhattan_distance(puzzle):
    return calculate_heuristic(puzzle,
                                lambda r, tr, c, tc: abs(tr - r) + abs(tc - c),
                                lambda t: t)


def heuristic_misplaced_tiles(puzzle):
    return calculate_heuristic(puzzle,
                                lambda r, tr, c, tc: misplaced_tiles_count(puzzle),
                                lambda t: t)


def misplaced_tiles_count(puzzle):
    count = 0
    for row in range(4):
        for col in range(4):
            if puzzle.get_value(row, col) != GOAL_STATE[row][col]:
                count += 1
    return count


def main():
    input_matrix = input("\nEnter the initial matrix state (For example: 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 0): ")
    input_matrix = input_matrix.split()
    p = FifteenPuzzle()
    p.initialize_matrix(input_matrix)
    perform_search(p)


def perform_search(p):
    print("\n\nBest-First Search")
    print("\n\nHeuristic 1: Euclidean distance")
    print(p)
    path, count = p.solve_using_best_first_search(heuristic_euclidean_distance)
    if path != []:
        path.reverse()
        for i in path:
            print(i.adj_matrix)
        step_count[1][1] = step_count[1][1] + len(path)
        print("\nAverage Steps: ", len(path))
    else:
        print("Search aborted after exploring", count - 1, "states")

    print("\n\nHeuristic 2: Manhattan distance ")
    print(p)
    path, count = p.solve_using_best_first_search(heuristic_manhattan_distance)
    if path != []:
        path.reverse()
        for i in path:
            print(i.adj_matrix)
        step_count[1][0] = step_count[1][0] + len(path)
        print("\nAverage Steps: ", len(path))
    else:
        print("Search aborted after exploring", count - 1, "states")

    print("\n\nHeuristic 3: Misplaced tiles")
    print(p)
    path, count = p.solve_using_best_first_search(heuristic_misplaced_tiles)
    if path != []:
        path.reverse()
        for i in path:
            print(i.adj_matrix)
        step_count[1][2] = step_count[1][2] + len(path)
        print("\nAverage Steps: ", len(path))
    else:
        print("Search aborted after exploring", count - 1, "states")

    print("\n\nA* Search")
    print("\n\nHeuristic 1: Euclidean distance ")
    print(p)
    path, count = p.solve_using_astar(heuristic_euclidean_distance)
    if path != []:
        path.reverse()
        for i in path:
            print(i.adj_matrix)
        step_count[0][1] = step_count[0][1] + len(path)
        print("\nAverage Steps: ", len(path))
    else:
        print("\nSearch aborted after exploring", count - 1, "states")

    print("\n\nHeuristic 2: Manhattan distance ")
    print(p)
    path, count = p.solve_using_astar(heuristic_manhattan_distance)
    if path != []:
        path.reverse()
        for i in path:
            print(i.adj_matrix)
        step_count[0][0] = step_count[0][0] + len(path)
        print("\nAverage Steps: ", len(path))
    else:
        print("\nSearch aborted after exploring", count - 1, "states")

    print("\n\nHeuristic 3: Misplaced tiles ")
    print(p)
    path, count = p.solve_using_astar(heuristic_misplaced_tiles)
    if path != []:
        path.reverse()
        for i in path:
            print(i.adj_matrix)
        step_count[0][2] = step_count[0][2] + len(path)
        print("\nAverage Steps: ", len(path))
    else:
        print("\nSearch aborted after exploring", count - 1, "states")


if __name__ == "__main__":
    main()



Enter the initial matrix state (For example: 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 0): 1 2 3 4 5 6 7 8 9 10  11 12 0 13 14 15


Best-First Search


Heuristic 1: Euclidean distance
1 2 3 4
5 6 7 8
9 10 11 12
0 13 14 15

[[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12], [13, 0, 14, 15]]
[[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12], [13, 14, 0, 15]]
[[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12], [13, 14, 15, 0]]

Average Steps:  3


Heuristic 2: Manhattan distance 
1 2 3 4
5 6 7 8
9 10 11 12
0 13 14 15

[[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12], [13, 0, 14, 15]]
[[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12], [13, 14, 0, 15]]
[[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12], [13, 14, 15, 0]]

Average Steps:  3


Heuristic 3: Misplaced tiles
1 2 3 4
5 6 7 8
9 10 11 12
0 13 14 15

[[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12], [13, 0, 14, 15]]
[[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12], [13, 14, 0, 15]]
[[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12], [13, 14, 15, 0]]

Average Steps:  3


A* Search


